In [1]:
import json, os
from openai import OpenAI

source_data_path = "../data/raw_data/"

# filename = "summary_zho_Territorial_disputes_in_the_South_China_Sea_100.json"
filename = "summary_eng_Territorial_disputes_in_the_South_China_Sea_100.json"
source_filename = source_data_path + filename

des_data_path = ""
des_filename = des_data_path + "test_QnA_" + filename

# Load articles
with open(source_filename, "r", encoding="utf-8") as f:
    articles = json.load(f)

# Load questions
with open("formated_output_questions.json", "r", encoding="utf-8") as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [2]:
def get_answers(article, questions, model):
    article_title = article.get("title")
    article_content = article.get("body")
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():

    pre = "You are an AI trained to understand articles and generate concise answers to multiple-choice questions based on the content. \
        Please read the following article carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \
                        If your answer is E: None of the above, please provide the explaination.\n\n"

    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model, messages=[{"role": "user", "content": prompt}], temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [3]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ";":
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(";")}


results = []
iteration = 0
max_iteration = len(articles)

for article in articles:
    # article_content = article['body']

    answers = get_answers(article, questions, model="gpt-4o")

    pure_answers = answers.choices[0].message.content
    print(pure_answers)

    # convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
    processed_answers = convert_string_to_dict(pure_answers)
    article["answers"] = processed_answers

    results.append(article)
    iteration += 1
    print(f"Processed {iteration} out of {max_iteration}")


# Save results to a JSON file
with open(des_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Proceeding article World News | Philippines Says China Coast Guard Seized Food Dropped by Plane for Filipino Forces in Disputed Sea | LatestLY
1D;2C;3B;4B;5A;6E;7A;8A;9A;10A
Processed 1 out of 100
Proceeding article Turkey's Top Diplomat Visits China, Planning Stops in Uyghur Genocide Region
1E;2E;3E;4E;5E;6E;7E;8E;9E;10E

Explanation: The questions provided do not relate to the content of the article about Turkish Foreign Minister Hakan Fidan's visit to China and the Uyghur genocide. The questions seem to be about a different event involving the South China Sea and the Philippines, which is not mentioned in the article. Therefore, the correct answer for all questions is E: None of the above.
Processed 2 out of 100
Proceeding article Phl, U.S., Japan coast guards vow to keep SCS safe, free
1E;2E;3A;4A;5D;6C;7B;8A;9C;10A

Explanation for 1E: None of the provided choices accurately describe the event discussed in the article. The article focuses on the collaboration between the Philippin